# Aplikasi Sampling Penutup Lahan
## Bagian 2. Sampling tool

2021-08-08

Tool untuk assign class pada setiap titik sampel.

### Packages

In [2]:
import geopandas as gpd
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from IPython.display import display
import ee
import geemap
import numpy as np

In [3]:
ee.Initialize()

### Dataset

In [4]:
# read generated random samples shapefiles downloaded from Google Drive
randsamp = gpd.read_file('samples/Aug-06-2021_Borneo_2016_stratifiedsamples_15_1000.shp')

# create ID column
randsamp['ID'] = np.arange(len(randsamp))

# landsat image as background
landsat = ee.Image('users/kfaisal/LCMS_Borneo_2016/L8_Borneo_2016_int')

### Sampling tool

In [5]:
# Create widgets
fill_id = widgets.BoundedIntText(0,0,1000,1)
selection_fnf = widgets.Select(options = ['Forest', 'Non-forest'])

def query(ID, Class, randsamp):
    randsamp.loc[randsamp['ID'] == ID, 'Class'] = Class
    sel_randsamp = randsamp[randsamp['ID'] == ID]
    
    # convert geodtaframe to ee object
    ee_randsamp = geemap.geopandas_to_ee(sel_randsamp)
    
    # display map
    Map = geemap.Map()
    # add Landsat layer to map
    Map.addLayer(landsat, {'bands': ['B5','B6','B7'],
                           'min': 40,
                           'max': 4000},
                'Landsat image')
    Map.addLayer(ee_randsamp)
    Map.centerObject(ee_randsamp, 10)
    
    # print selected object
    print(sel_randsamp)
    return Map

interact(query, ID = fill_id, Class = selection_fnf, randsamp = fixed(randsamp))

interactive(children=(BoundedIntText(value=0, description='ID', max=1000), Select(description='Class', options…

<function __main__.query(ID, Class, randsamp)>

### Check results

In [7]:
randsamp.head()

,cluster,geometry,ID,Class
0,0,POINT (117.46847 2.50962),0,Forest
1,0,POINT (116.71568 6.11366),1,Forest
2,0,POINT (117.05075 4.67546),2,Forest
3,0,POINT (116.12279 3.72864),3,Forest
4,0,POINT (114.69716 3.53730),4,Non-forest


### Export results

In [22]:
# Export to Asset, later used as machine-learning classification input in GEE
from datetime import date
today = date.today()
todaydate = today.strftime("%b-%d-%Y")

# define output file name
exportName = 'Borneo'

# Convert geodataframe to ee object
ee_export = geemap.geopandas_to_ee(randsamp)

exportTask = ee.batch.Export.table.toAsset(
    collection = ee_export,
    description = todaydate + '_' + exportName + '_forestCoverSamples',
    assetId = 'users/gemasaktiadzan/2021-08-08_Borneo_forestCoverSamples')
exportTask.start()